In [5]:
!pip install trax

In [6]:
import trax.layers as tl
from nltk.corpus import stopwords,twitter_samples
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from string import punctuation
import re
from sklearn.model_selection import train_test_split
import  trax.fastmath.numpy as np  
from trax.supervised.training import TrainTask,EvalTask,Loop
import itertools
import trax
import numpy
import random as rnd

In [7]:
positive_tweet=twitter_samples.strings('positive_tweets.json')
positive_label=[1]*len(positive_tweet)
negative_tweet=twitter_samples.strings('negative_tweets.json')
negative_label=[0]*len(negative_tweet)
len(positive_tweet),len(negative_tweet)

(5000, 5000)

In [8]:
all_data=positive_tweet+negative_tweet
all_label=positive_label+negative_label

In [9]:
X_train, X_test, y_train, y_test=train_test_split(all_data,all_label,test_size=0.2,random_state=66)
X_test, X_val, y_test, y_val=train_test_split(X_test,y_test,test_size=0.5,random_state=66)
print('Type of X_train: ',type(X_train))
print('Type of X_train: ',(type(y_train)))
print('X_train shape',len(X_train))
print('X_test shape',len(X_test))
print('X_val shape',len(X_val))
print('Sample From Data',X_train[0])
print('Sample From Label',y_train[0])

Type of X_train:  <class 'list'>
Type of X_train:  <class 'list'>
X_train shape 8000
X_test shape 1000
X_val shape 1000
Sample From Data @ReetAwwsum Great! Did you reserve for Windows 10? :) ^MM
Sample From Label 1


In [10]:
def preprocess_tweete(corbus,tokenizer,steeming,stop_words):
    clean_corbus=[]
    for tweet in  corbus:
        # remove old style retweet text "RT"
        tweet = re.sub(r'^RT[\s]+', '',tweet )

        # remove hyperlinks
        tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)

        # remove hashtags
        # only removing the hash # sign from the word
        tweet = re.sub(r'#', '', tweet)
        
        tokenzie_tweet=tokenizer(tweet)
        
        clean_tweet=[]
        
        for token in tokenzie_tweet:
            if token not in stop_words and token not in punctuation:
                stem_word=steeming(token)
                clean_tweet.append(stem_word)

        
        
        clean_corbus.append(clean_tweet)
    return clean_corbus
        

In [11]:
tokenize=TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
steming=PorterStemmer()
stop_words=stopwords.words('english')
print('Sample From StopWords: ', stop_words[:10])

Sample From StopWords:  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [12]:
X_train=preprocess_tweete(X_train,tokenize.tokenize,steming.stem,stop_words)
X_val=preprocess_tweete(X_val,tokenize.tokenize,steming.stem,stop_words)
X_test=preprocess_tweete(X_test,tokenize.tokenize,steming.stem,stop_words)
print('Sample from data',X_train[0])

Sample from data ['great', 'reserv', 'window', '10', ':)', 'mm']


In [13]:
def get_vocab(tweets):
    vocab={'pad':0,'unknow':1}
    for tweet in tweets:
        for word in tweet:
            if word not in vocab:
                vocab[word]=len(vocab)
    return vocab
vocab=get_vocab(X_train)
print('The Size of vocabulary is: ',len(vocab))

The Size of vocabulary is:  9161


In [14]:
def tweet_to_tensor(tweet,vocab):
    tensor=[]
    for word in tweet:
        tensor.append(vocab.get(word,vocab['unknow']))
    return tensor

In [15]:
print('X_val smaple',X_val[0])
print('tensor',tweet_to_tensor(X_val[0],vocab))

X_val smaple ['would', 'love', 'see', 'w', 'nick', 'pick', 'one', 'waist', 'tv', 'time', 'never', 'work', 'anyway', ':(']
tensor [528, 38, 92, 1717, 1, 160, 137, 8548, 1409, 249, 205, 461, 501, 11]


In [16]:
def generator(data,label,batch_size=256):
    data_size=len(data)
    patch=[]
    Input=[]
    Output=[]
    idx=0
    while True:
        if idx >=data_size:
            idx=0
        tensor=tweet_to_tensor(data[idx],vocab)
        patch.append(tensor)
        Output.append(label[idx])
        idx+=1
        
        if len(patch)==batch_size:
            max_len=int(2**numpy.ceil(np.log2(max([len(i) for i in patch]))))
            for tensor in patch:
                tensor=tensor+([vocab['pad']]*(max_len-len(tensor)))
                Input.append(tensor)
            Input=np.array(Input)
            Output=np.array(Output)
            Mask=np.ones_like(Output)
            yield Input,Output,Mask
            Input,Output,patch=[],[],[]
        
        
        

In [17]:
def model(n_dimension,num_class,vocab):
    vocab_size=len(vocab)
    
    model=tl.Serial([tl.Embedding(vocab_size,n_dimension)
               ,tl.LSTM(n_dimension),
                tl.Mean(1),
               tl.Dense(num_class),
               tl.LogSoftmax()])
    
    return model

In [18]:
print(model(256,2,vocab))

Serial[
  Embedding_9161_256
  LSTM_256
  Mean
  Dense_2
  LogSoftmax
]


In [19]:
# PLEASE, DO NOT MODIFY OR DELETE THIS CELL
from trax.supervised import training

def get_train_eval_tasks(train_generator,val_generator):
    
    rnd.seed(271)

    train_task = training.TrainTask(
        labeled_data=train_generator,
        loss_layer=tl.WeightedCategoryCrossEntropy(),
        optimizer=trax.optimizers.Adam(0.01),
        n_steps_per_checkpoint=10,
    )

    eval_task = training.EvalTask(
        labeled_data=val_generator,        
        metrics=[tl.WeightedCategoryCrossEntropy(), tl.WeightedCategoryAccuracy()],
    )
    
    return train_task, eval_task
    

train_task, eval_task = get_train_eval_tasks(generator(X_train,y_train),generator(X_val,y_val))


In [20]:
dir_path = './model/'

n_step=int((len(X_train)/256))*3

def trainig (classifier,n_step,train_task,eval_task):
    train=training.Loop(classifier,train_task,eval_tasks=eval_task, output_dir=dir_path)
    train.run(n_step)
    return train

In [21]:
num_class=np.sum(np.unique(np.array(y_train)))+1
n_dimension=256
classifier = model(n_dimension,num_class,vocab)
print('Show Sup_Layer in My Model\n\n',classifier)

Show Sup_Layer in My Model

 Serial[
  Embedding_9161_256
  LSTM_256
  Mean
  Dense_2
  LogSoftmax
]


In [23]:
#classifier=model(256,2,vocab)
model=trainig (classifier,n_step,train_task,[eval_task])

TypeError: unsupported operand type(s) for ==: 'DeviceArray' and 'tuple'

In [35]:
def accuracy(model,X_test,y_test):
    X_test,y_test,Mask=next(generator(X_test,y_test,batch_size=len(X_test)))
    prediction=model.eval_model(X_test)
    compresion=np.argmax(prediction,axis=1)==y_test
    accuracy=np.sum(compresion)/len(X_test)
    return accuracy

In [37]:
print('The Accuracy Of Model Is ',accuracy(model,X_test,y_test))

The Accuracy Of Model Is  0.98700005
